# Anwendung von maschinellem Lernen auf den KHK_Klassifikation.csv Datensatz

## Praktische Demonstration für verschiedene machine Learning Modelle

### Tim Bleicher, Linus Pfeifer

Dieses Jupyter Notebook demonstriert die Anwendung von verschiedenen Machine Learning Modellen auf den KHK_Klassifikation.csv Datensatz. 

# Inhaltsverzeichnis

## 1. Einbindung der Daten
- **1.1 Explorative Analyse der Daten**

## 2. PCA-Dimensionsreduzierung zur Visualisierung und Analyse der Daten
- **Funktionsweise von PCA**
- **Lässt sich aus den PCA-Daten eine potentielle gute Separierbarkeit der Klassen ablesen?**

## 3. Anwendung verschiedener Klassifikationsverfahren
- **Definition und Datenvorbereitung**
- **3.1 Logistische Regression**
  - 3.1.1 Modell definieren und trainieren
  - 3.1.2 Modell testen
- **3.2 Entscheidungsbäume**
  - 3.2.1 Klassische Entscheidungsbäume
  - 3.2.2 Bagging in Form von Random Forest
  - 3.2.3 Boosting in Form von AdaBoost
  - 3.2.4 Stacking
- **3.3 k-Nearest-Neighbor**
  - 3.3.1 k-Nearest-Neighbor mit euklidischer Metrik
  - 3.3.2 k-Nearest-Neighbor mit Manhattan Metrik
  - 3.3.3 k-Nearest-Neighbor mit Minkowski Metrik (p = 3)
- **3.4 Support Vector Machine**
- **3.5 Neuronales Netz**

## 4. Bedeutung der einzelnen Features
- **4.1 Feature-Bedeutung von PCA**
- **4.2 Feature-Bedeutung für Random Forest**
- **4.3 Feature-Bedeutung für SVM**

## 5. Feature-Engineering
- **5.1 Generieren der PCA-Hauptkomponenten-Daten**
- **5.2 Testen des Feature-Engineering auf k-Nearest-Neighbor mit Manhattan Metrik**
- **5.3 Testen des Feature-Engineering auf einem klassischen Entscheidungsbaum**


## 1. Einbindung der Daten

Zu beginn des Projekts werden die Daten zunächst geladen um diese im anschluss analysieren und nutzen zu können.

In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
import plotly.graph_objects as go
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
data = pd.read_csv('KHK_Klassifikation.csv', sep=',')

## 1.1 Explorative Analyse der Daten 

Die explorative Datenanalyse (EDA) ist ein Ansatz zur Untersuchung von Datensätzen, bei dem zunächst deren Hauptmerkmale visuell und statistisch beschrieben werden – oft noch ohne eine konkrete Hypothese. Ziel ist es, ein erstes Verständnis für Struktur, Muster, Ausreißer, Verteilungen und potenzielle Zusammenhänge in den Daten zu bekommen (vgl. https://www.ibm.com/think/topics/exploratory-data-analysis).

### 📄 Beschreibung der Attribute im Datensatz

| Attribut      | Beschreibung |
|---------------|-------------|
| **Alter** | Alter der Patientin oder des Patienten in Jahren. |
| **Geschlecht** | Geschlecht der Person: <br>`M` steht für männlich, `F` für weiblich. |
| **Blutdruck** | Systolischer Blutdruck in mmHg (Millimeter Quecksilbersäule), gemessen im Ruhezustand. Werte ab 140 gelten in der Regel als erhöhter Blutdruck. (vgl. https://www.visomat.de/blutdruck-normalwerte/)|
| **Chol** | Gesamtcholesterin im Blut in mg/dL (Milligramm pro Deziliter). Erhöhte Werte (>190 mg/dL) können ein Risiko für Herz-Kreislauf-Erkrankungen darstellen. (vgl. https://www.cholesterinspiegel.de/auffaellige-cholesterinwerte/) |
| **Blutzucker** | Nüchtern-Blutzuckerwert: <br>`0` = Normaler Blutzucker <br>`1` = Erhöhter Blutzucker (möglicher Hinweis auf Diabetes oder Prädiabetes). |
| **EKG** | Ergebnis des Ruhe-EKGs. Mögliche Kategorien: <br>- `Normal` = unauffälliger Befund <br>- `ST` = ST-Streckensenkung (Hinweis auf Belastungsischämie) <br>- `LVH` = Linksventrikuläre Hypertrophie (Herzmuskelvergrößerung). |
| **HFmax** | Maximale Herzfrequenz (in Schlägen pro Minute), die während eines Belastungstests erreicht wurde. Sehr grobe Faustregel: HFmax = 220 - Lebensalter (vgl. https://www.germanjournalsportsmedicine.com/archive/archive-2010/heft-12/die-maximale-herzfrequenz/) |
| **AP** | Angina Pectoris bei Belastung: <br>`N` = Keine Symptome <br>`Y` = Auftreten von Angina Pectoris (Brustschmerzen unter Belastung), möglicher Hinweis auf Durchblutungsstörungen des Herzens. |
| **RZ** | Rückgang (bzw. Veränderung) der ST-Strecke während eines Belastungs-EKGs in **mm**. <br> Positive Werte deuten auf eine **ST-Streckensenkung** hin, was auf eine mögliche **Ischämie des Herzmuskels** (z. B. bei KHK) hindeuten kann. <br> Negative Werte können als **ST-Streckenhebung** interpretiert werden – diese können je nach klinischem Zusammenhang normal, unspezifisch oder auch pathologisch sein (z. B. bei Infarkten oder Perikarditis). <br> In der Regel gilt: Je größer der **absolute Betrag**, desto auffälliger der Befund. |
| **KHK** | **Zielvariable** – Diagnose einer koronaren Herzkrankheit: <br>`0` = Keine KHK <br>`1` = KHK nachgewiesen (positives Ergebnis). |



**Erster Blick auf die Daten:**  
Zuerst wird eine Kopie des Datensatzes erstellt und ein erster Blick auf die obersten Zeilen geworfen.
Dies dient dazu einen ersten groben Überblick über die Daten zu bekommen. 

In [ ]:
# Copy of the original dataset
df = data.copy()

# Display the first few rows
display(df.head())

**Allgemeine Informationen:**  
Mit `df.info()` erhält man einen Überblick über die Spalten, Datentypen und Anzahl fehlender Werte. Das ist wichtig, um zu verstehen, welche Features numerisch oder kategorisch sind.

In [ ]:
# General information about the dataset
display(df.info())

**Statistische Kennzahlen:**  
Diese Übersicht zeigt zentrale Lage- und Streuungsmaße (z. B. Mittelwert, Standardabweichung) für alle numerischen Spalten. Das hilft, Ausreißer oder ungewöhnliche Verteilungen frühzeitig zu erkennen.

In [ ]:
# Statistical overview of numerical features
display(df.describe())

**Kategoriale Merkmale analysieren:**  
Nun wird die Häufigkeit der Werte in allen kategorialen Spalten angesehen. So erkennt man dominante Klassen und mögliche Ungleichgewichte.

In [ ]:
# Frequency of values for categorical features
for col in df.select_dtypes(include=['object']).columns:
    print(f"\nValue distribution for '{col}':")
    print(df[col].value_counts())

**Fehlende Werte prüfen:**  
Hier wird analysiert, in welchen Spalten Daten fehlen. Dies ist entscheidend für die spätere Datenbereinigung oder Modellierung.


In [ ]:
# Missing values
print("\nMissing values per column:")
print(df.isnull().sum())

**Doppelte Einträge identifizieren:**  
Es wird geprüft, ob es doppelte Zeilen gibt – diese sollten bei Bedarf entfernt werden, um Verzerrungen zu vermeiden.


In [ ]:
# Check for duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())

**Zielvariable analysieren:**  
Ein schneller Blick auf die Verteilung der Zielgröße (KHK: koronare Herzkrankheit). So sieht man z. B., ob die Klassen unausgeglichen sind.


In [ ]:
# Distribution of the target variable (CHD)
print("\nDistribution of the target variable 'CHD':")
print(df["KHK"].value_counts())

**Boxplots zur Übersicht:**  
Boxplots geben einen schnellen Überblick über die Verteilung numerischer Merkmale inkl. Ausreißer. Dies ist besonders hilfreich zum Erkennen von Extremwerten.


In [ ]:
import plotly.express as px

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
numerical_cols_filtered = [
    col for col in numerical_cols if df[col].nunique() > 2]

for col in numerical_cols_filtered:
    fig = px.box(df, y=col, points="all",
                 title=f"Boxplot: {col}", template="plotly_white")
    fig.update_layout(yaxis_title=col)
    fig.show()

**Histogramme zur Dichteverteilung:**  
Diese Plots zeigen, wie sich die Werte in den numerischen Spalten verteilen. Zusätzlich ist oben ein Boxplot integriert.

In [ ]:
for col in numerical_cols_filtered:
    fig = px.histogram(df, x=col, nbins=20, marginal="box",
                       title=f"Histogramm: {col}", template="plotly_white", color_discrete_sequence=["steelblue"])
    fig.update_layout(xaxis_title=col, yaxis_title="Häufigkeit")
    fig.show()

**Boxplots nach KHK-Klassen:**  
Hier wird analysiert, ob sich bestimmte numerische Merkmale in Abhängigkeit von der Zielvariable signifikant unterscheiden. Das kann Hinweise auf relevante Prädiktoren liefern.

Wichtig zu erwähnen ist, dass viele Chol Werte 0 sind, was auf fehlende Daten hinweist. Daher wurden diese Werte herausgefiltert, um die Statistiken nicht zu verfälschen.


In [ ]:
numerical_cols_khk = [
    col for col in numerical_cols
    if df[col].nunique() > 2 and col != "KHK" and col != "Blutzucker"
]

for col in numerical_cols_khk:
    df_plot = df.copy()

    # For the Chol column: exclude values with 0
    if col == "Chol":
        df_plot = df_plot[df_plot["Chol"] != 0]

    fig = px.box(df_plot, x="KHK", y=col, color="KHK",
                 title=f"{col} by CHD class", template="plotly_white", points="all")
    fig.update_layout(xaxis_title="CHD (0 = No, 1 = Yes)", yaxis_title=col)
    fig.show()

## 2. PCA-Dimensionsreduzierung zur Visualisierung und Analyse der Daten 

### Funktionsweise von PCA
Die Hauptkomponentenanalyse (PCA) dient der Dimensionsreduktion eines Datensatzes. Dies ermöglicht beispielsweise verschiedene Analyse des gesamten Datensatzes (mit mehr als 3 Dimensionen), wobei die Ergebnisse durch die Dimensionsreduktion weiterhin visualisiert werden können.
Das Verfahren der PCA läuft nach folgendem Schema ab:

1. Berechnung des Mittelwerts und Zentrierung der Daten
2. Berechnung der Kovarianzmatrix
3. Berechnung der Eigenwerte und Eigenvektoren
4. Transformation der Daten

Damit die PCA korrekt funktioniert, muss zunächst von jeder Dimension der Mittelwert subtrahiert werden. Dieser Mittelwert entspricht dem Durchschnittswert jeder Dimension. Beispielsweise wird von allen $x$-Werten der Mittelwert $\overline{x}$ subtrahiert. Entsprechendes gilt für die anderen Dimensionen der Daten. Dadurch entsteht ein Datensatz mit einem Mittelwert von null.

Im nächsten Schritt wird die Kovarianzmatrix berechnet, welche die wechselseitigen Zusammenhänge zwischen den Merkmalen quantifiziert. Falls zwei Merkmale stark korrelieren, können diese in einer neuen Achse kombiniert werden.

Anschließend werden die Eigenwerte und Eigenvektoren der Kovarianzmatrix bestimmt. Die Eigenvektoren definieren die Richtungen der Hauptkomponenten, während die zugehörigen Eigenwerte die Bedeutung bzw. die Varianz der jeweiligen Eigenvektoren widerspiegeln.

Es folgt die eigentliche Dimensionsreduktion, indem nur diejenigen Eigenvektoren mit den größten Eigenwerten ausgewählt werden. Diese Eigenvektoren entsprechen den neuen Hauptachsen des Datensatzes.

Schließlich werden die Daten transformiert, indem die ursprüngliche Datenmatrix mit der Matrix der Eigenvektoren multipliziert wird. In dieser Matrix repräsentiert jede Spalte einen Eigenvektor.



In [ ]:
label_encoder = LabelEncoder()
categorical_columns = ['Geschlecht', 'EKG', 'AP']

for col in categorical_columns:
    # Encode categorical columns
    data[col] = label_encoder.fit_transform(data[col])

print(data.head())


In [ ]:
# Remove the target variable "KHK" before scaling
data_without_target = data.drop(columns=["KHK"], errors="ignore")

# Scale the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_without_target)

# PCA transformation with two principal components
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)

# Convert the PCA results into a DataFrame
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

# Interactive visualization
fig = px.scatter(df_pca, x='PC1', y='PC2', title='PCA Visualization of the Data', opacity=0.5)
fig.show()


In [ ]:
target = data["KHK"]
df_pca['KHK'] = target

df_pca['KHK'] = df_pca['KHK'].astype('category')

# Interactive visualization with colors based on KHK (0 = blue, 1 = red)
fig = px.scatter(
    df_pca,
    x='PC1',
    y='PC2',
    color='KHK',
    color_discrete_map={0: 'blue', 1: 'red'},  # Discrete color mapping
    title='PCA visualization colored by KHK class',
    opacity=0.5
)

# Adjust marker size and legend settings
fig.update_traces(marker=dict(size=5))
fig.update_layout(
    legend_title_text='KHK',  # Clearly label the legend
    showlegend=True           # Ensure the legend is displayed
)

# Show the plot
fig.show()

### Lässt sich aus den PCA-Daten eine potentielle gute Separierbarkeit der Klassen ablesen?

Es sind zwar einzelne "Flecken" zu sehen, die etwas konzentriertere Mengen an Punkten darstellen, jedoch ist es schwierig möglich hieraus verschiedene Klassen abzuleiten.
Dies liegt vor allem an der Reduktion der Vielzahl an Attributen auf nur 2 Hauptachsen, wobei dann doch zu viele Informationen bei der Darstellung verloren gehen.
Sind die Zusammenhänge von Attributen nicht-linear (wovon man in diesem Datensatz erstmal ausgehen kann), so gehen wichtige Informationen bei der PCA verloren, da die Abbildung auf den niedrigdimensionalen Raum lediglich linear ist (vgl. Skript ML_10_Dimensionsreduktion S.16-17).


## 3. Anwendung verschiedener vorgestellter Klassifikationsverfahren

#### Definition und Datenvorbereitung

Zunächst werden die kategorialen und numerischen Merkmale des Datensatzes definiert. Anschließend erfolgt die Vorbereitung der Daten für ein Machine-Learning-Modell. Dazu gehören die Auswahl und Umordnung der Merkmale, die Umwandlung kategorialer Variablen mittels Label-Encoding (vgl. https://kantschants.com/complete-guide-to-encoding-categorical-features), die Standardisierung der numerischen Variablen sowie die Aufteilung in Trainings- und Testdaten.

In [ ]:
# Define categorical and numerical columns
categorical_features = ["Geschlecht", "EKG", "AP"]
numerical_features = ["Alter", "Blutdruck", "Chol", "Blutzucker", "HFmax", "RZ"]

# Select target variable and features
X = data[categorical_features + numerical_features].copy()
y = data["KHK"]

# Reorder features to match the desired order
desired_order = ["Alter", "Geschlecht", "Blutdruck", "Chol", "Blutzucker", "EKG", "HFmax", "AP", "RZ"]
X = X[desired_order]

# Apply Label Encoding to categorical features
label_encoders = {}  # Store LabelEncoder objects
for col in categorical_features:
    label_encoders[col] = LabelEncoder()
    X[col] = label_encoders[col].fit_transform(X[col])

# Standardize numerical features
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 3.1 Logistische Regression
Logistische Regression ist ein statistisches Modell, das den natürlichen Logarithmus der Chancen eines Ereignisses als lineare Kombination einer oder mehrerer unabhängiger Variablen modelliert.
In der Regressionsanalyse schätzt die logistische Regression die Parameter dieses Modells, typischerweise in Szenarien mit einer binären Zielvariablen (z. B. 0 oder 1) (vgl. https://en.wikipedia.org/wiki/Logistic_regression).

Logistische Regression passt gut zu dem Datensatz, da KHK binär ist, der Datensatz gemischte Merkmale enthält und das Modell wahrscheinlichkeiten für KHK einfach und interpretierbar berechnet.

#### 3.1.1 Modell definieren und trainieren

Das beschriebene logistische Regressionsmodell wird erstellt und mit den Trainingsdaten trainiert, um KHK als binäre Zielvariable vorherzusagen.

In [ ]:
# Logistic Regression for binary classification

# Create pipeline with preprocessing and logistic regression
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)


#### 3.1.2 Modell testen

Das trainierte Modell wird auf den Testdaten angewendet. Die Auswertung erfolgt anhand der **Genauigkeit (Accuracy)** und eines **Classification Reports**, der die wichtigsten Metriken zur Bewertung der Vorhersagequalität enthält:

| **Metrik**     | **Beschreibung**                                                                 |
|----------------|-----------------------------------------------------------------------------------|
| **Accuracy**   | Anteil korrekt klassifizierter Beispiele an allen Beispielen                     |
| **Precision**  | Anteil korrekt positiver Vorhersagen an allen als positiv vorhergesagten Fällen (vgl. https://www.v7labs.com/blog/precision-vs-recall-guide)  |
| **Recall**     | Anteil korrekt erkannter positiver Fälle an allen tatsächlichen positiven Fällen (vgl. https://www.v7labs.com/blog/precision-vs-recall-guide) |
| **F1-Score**   | Harmonisches Mittel aus Precision und Recall (balanciert beide Metriken) (vgl. https://www.v7labs.com/blog/f1-score-guide)       |

Diese Metriken geben gemeinsam ein gutes Bild darüber, wie zuverlässig das Modell bei der KHK-Klassifikation arbeitet.

In [ ]:
# Make predictions
y_pred_log_reg = model.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred_log_reg)
classification_rep = classification_report(y_test, y_pred_log_reg)

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(classification_rep)


Die logistische Regression erzielt eine Genauigkeit von 76 % und zeigt insgesamt eine ausgewogene Leistung.
Besonders bei der Erkennung von KHK-Fällen (Klasse 1) schneidet das Modell gut ab, mit einer Präzision von 82 % und einem Recall von 76 %.
Die Klasse 0 (keine KHK) wird mit etwas geringerer Präzision (69 %) erkannt, aber mit solider Sensitivität (77 %).

Insgesamt liefert das Modell stabile und gut ausbalancierte Vorhersagen und eignet sich als verlässlicher Basisansatz für die Klassifikation von KHK.

### 3.2 Entscheidungsbäume

Ein Entscheidungsbaum ist ein geordneter, gerichteter Baum, der Entscheidungsregeln in einer hierarchischen Struktur darstellt
Jeder Knoten repräsentiert eine Bedingung oder Entscheidungsregel basierend auf einem Merkmal des Datensatzes (vgl. https://de.wikipedia.org/wiki/Entscheidungsbaum).
Die Blattknoten am Ende des Baums geben die Vorhersage oder Klassenzugehörigkeit aus (in unserem Fall KHK = 0 oder 1).
Die Struktur ermöglicht es, Daten schrittweise entlang dieser Bedingungen aufzuteilen, um zu einer fundierten Entscheidung zu gelangen.

#### 3.2.1 Klassische Entscheidungsbäume

Der klassische Entscheidungsbaum funktioniert wie in [3.2](#32-entscheidungsbäume) beschrieben. Es werden keine Ensemble-Methoden verwendet, sondern ein einzelner Baum erstellt, der rekursiv durch Splits an Knotenpunkten aufgebaut wird.

Auch hier wird die Klassifikation wieder an den schon beschriebenen Metriken gemessen.

In [ ]:
# Train a Decision Tree Classifier
clf_tree = DecisionTreeClassifier(random_state=42)
clf_tree.fit(X_train, y_train)

# Make predictions
y_pred_tree = clf_tree.predict(X_test)

# Calculate accuracy
accuracy_tree = accuracy_score(y_test, y_pred_tree)
classification_rep_tree = classification_report(y_test, y_pred_tree)

# Print results
print(f"Model accuracy: {accuracy_tree:.2f}")
print(classification_rep_tree)


Der klassische Entscheidungsbaum erreicht eine Genauigkeit von 64 % und bleibt damit deutlich hinter der logistischen Regression zurück.
Während KHK-Fälle (Klasse 1) mit einer Präzision von 70 % noch relativ gut erkannt werden, zeigt das Modell bei der Klasse 0 (keine KHK) Schwächen, vor allem bei der Präzision mit nur 56 %.

Für den vorliegenden Datensatz liefert der klassische Entscheidungsbaum somit nur begrenzte Vorhersagequalität und generalisiert schlechter als das lineare Modell.

#### 3.2.2 Bagging in Form von Random Forest
Random Forest ist ein Modell, das viele Entscheidungsbäume kombiniert. Jeder Baum wird auf zufälligen Daten und Merkmalen trainiert.
Die Vorhersagen der Bäume werden am Ende zusammengefasst (beispielsweise per Mehrheitsentscheid). Dadurch wird das Modell stabiler und genauer als ein einzelner Baum (vgl. https://www.ibm.com/de-de/think/topics/random-forest).

In [ ]:
# Train a Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred_random_forest = clf.predict(X_test)

# Calculate accuracy
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
classification_rep = classification_report(y_test, y_pred_random_forest)

# Print results
print(f"Model accuracy: {accuracy_random_forest:.2f}")
print(classification_rep)


Der Random Forest erzielt eine Genauigkeit von 73 % und liegt damit leicht unterhalb der logistischen Regression, aber deutlich über dem einfachen Entscheidungsbaum.
Die Klasse 1 (KHK) wird mit einer Präzision von 78 % und einem Recall von 75 % zuverlässig erkannt. Auch Klasse 0 (keine KHK) erreicht solide Werte mit 67 % Präzision und 71 % Recall.
Insgesamt zeigt das Modell eine ausgewogene Leistung mit guter Generalisierungsfähigkeit.

Daddurch, dass viele Entscheidungsbäume kombiniert werden, kann Random Forest wohl komplexere Zusammenhänge gut erfassen.

#### 3.2.3 Boosting in Form von AdaBoost

Adaptive Boosting ist einer der bekanntesten und breitesten Boosting Algorithmen.
Mehrere schwache Modelle werden nacheinander trainiert, wobei jedes neue Modell gezielt die Fehler der vorherigen korrigiert, indem falsch klassifizierte Daten stärker gewichtet werden. Die endgültige Vorhersage entsteht durch ein gewichtetes Mehrheitsvotum(vgl. Skript ML_08_Modelle-V S.26-34).

In [ ]:
# Define base estimator for AdaBoost
base_estimator = DecisionTreeClassifier(max_depth=1)

# Train AdaBoost model with specified parameters
adaboost_model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,
    learning_rate=0.3,
    random_state=42
)

adaboost_model.fit(X_train, y_train)

# Make predictions
y_pred_ada = adaboost_model.predict(X_test)

# Calculate accuracy
accuracy_random_forest = accuracy_score(y_test, y_pred_ada)  # Note: variable name might be misleading
classification_rep = classification_report(y_test, y_pred_ada)

# Print results
print(f"Model accuracy: {accuracy_random_forest:.2f}")
print(classification_rep)


Das AdaBoost-Modell erreicht eine Genauigkeit von 77 % und liefert damit das (in der chronologischen Reihenfolge des Notebooks) bislang beste Ergebnis unter den getesteten Verfahren.
Die Klasse 0 (keine KHK) wird mit einer Präzision von 70 % und einem Recall von 78 % solide erkannt, während die Klasse 1 (KHK) mit 83 % Präzision und 76 % Recall besonders zuverlässig vorhergesagt wird.

Durch das schrittweise Lernen aus Fehlern früherer Modelle gelingt AdaBoost eine gut ausbalancierte Klassifikation, die sowohl Präzision als auch Sensitivität berücksichtigt. Im Vergleich zur logistischen Regression und zum Random Forest schneidet AdaBoost leicht besser ab.

#### 3.2.4 Stacking

Stacking kombiniert verschiedene Modelle, die jeweils unterschiedliche Bereiche der Merkmalsverteilung gut abdecken, und nutzt deren Vorhersagen gemeinsam, um die Gesamtvorhersage zu verbessern(vgl. Skript ML_08_Modelle-V S.40-42).
Es ist besondern gut anzuwenden, wenn die Einzelmodelle ausreichend verschieden sind.

In [ ]:
# Base models: KNN, SVM, and Logistic Regression
base_estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),  # KNN with 5 neighbors
    ('svc', SVC(kernel='linear', random_state=42)),  # SVM with a linear kernel
    ('logreg', LogisticRegression(random_state=42))  # Logistic Regression
]

# Final model (meta-model)
final_estimator = LogisticRegression()

# Create StackingClassifier with base models and final estimator
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=final_estimator)

# Train the stacking model
stacking_model.fit(X_train, y_train)

# Make predictions
y_pred_stack = stacking_model.predict(X_test)

# Calculate accuracy
accuracy_stack = accuracy_score(y_test, y_pred_stack)
classification_rep = classification_report(y_test, y_pred_stack)

# Print results
print(f"Model accuracy: {accuracy_stack:.2f}")
print(classification_rep)


Das Stacking-Modell erreicht eine Genauigkeit von 76 % und liegt damit auf dem Niveau der logistischen Regression und knapp unter AdaBoost. Es kombiniert KNN, SVM und logistische Regression als Basis-Modelle und nutzt erneut eine logistische Regression als Meta-Modell, das aus den Vorhersagen der Basismodelle lernt.
Die Klasse 0 (keine KHK) wird mit 69 % Präzision und 77 % Recall erkannt, während Klasse 1 (KHK) mit 82 % Präzision und 75 % Recall klassifiziert wird.

Das Modell ist insgesamt ausgewogen, profitiert wohl von der Vielfalt der Basismodelle und zeigt, dass sich durch geschickte Kombination unterschiedlicher Ansätze eine solide Vorhersagequalität erzielen lässt.
Trotzdem ist das Modell minimal schlechter als das verhorige AdaBoost Modell.

### 3.3 k-Nearest-Neighbor
KNN ist ein einfacher Algorithmus, der neue Datenpunkte anhand der k ähnlichsten bekannten Punkte einordnet.
Er speichert nur die Trainingsdaten und trifft Entscheidungen bei der Vorhersage, basierend auf dem Abstand zu den nächsten Nachbarn (vgl. https://www.ibm.com/de-de/think/topics/knn).

#### 3.3.1 k-Nearest-Neighbor mit euklidischer Metrik

Es wird der direkcte Abstand zwischen zwei Punkten im Merkmalsraum gemessen.

In [ ]:
# Create k-NN model with k=10 and Euclidean distance metric
knn_model = KNeighborsClassifier(n_neighbors=10, metric='euclidean')

# Train the model
knn_model.fit(X_train, y_train)

# Make predictions
y_pred_knn = knn_model.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

# Print results
print(f"Model accuracy: {accuracy_knn:.2f}")
print(classification_rep_knn)


Das k-NN-Modell mit euklidischer Distanz liefert mit 78 % Genauigkeit das momentan beste Ergebnis (chronologisch anhand des Notebooks). Besonders auffällig ist die starke Leistung bei Klasse 0 (keine KHK): Mit einem Recall von 87 % erkennt das Modell sehr zuverlässig gesunde Fälle, auch wenn die Präzision bei 68 % liegt. Für Klasse 1 (KHK) ist die Präzision mit 88 % sogar noch höher, aber der Recall etwas niedriger bei 71 %.

Insgesamt zeigt sich ein leichtes Ungleichgewicht: Das Modell erkennt gesunde Patienten sehr sicher, neigt aber dazu, einige KHK-Fälle zu übersehen.
Dennoch ist die F1-Balance sehr gut und das Modell profitiert offenbar davon, dass k-NN bei gut strukturierten Daten (StandardScaler()) effektiv trennen kann (vgl. https://medium.com/@RobuRishabh/knn-k-nearest-neighbour-5ae18ae8e274).

#### 3.3.2 k-Nearest-Neighbor mit manhattan Metrik

Es wird der Abstand zwischen zwei Punkten als Summe der absoluten Differenzen der Merkmalswerte berechnet (Analogie zu rechteckigen Straßennetzen).
Dies ermöglicht höhere Robustheit gegen Außreißer

In [ ]:
# Create k-NN model with k=10 and Manhattan distance metric
knn_model = KNeighborsClassifier(n_neighbors=10, metric='manhattan')

# Train the model
knn_model.fit(X_train, y_train)

# Make predictions
y_pred_knn = knn_model.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

# Print results
print(f"Model accuracy: {accuracy_knn:.2f}")
print(classification_rep_knn)


Das k-NN-Modell mit Manhattan-Distanz erzielt mit 79 % Genauigkeit die bisher beste Gesamtleistung aller getesteten Modelle.

Für Klasse 0 (keine KHK) erreicht es einen sehr hohen Recall von 87 %, was bedeutet, dass gesunde Fälle fast vollständig erkannt werden. Die Präzision liegt bei akzeptablen 70%.
Klasse 1 (KHK) wird mit 89 % Präzision und 73 % Recall sehr zuverlässig vorhergesagt.

Die Manhattan-Metrik scheint in diesem Fall besonders gut zu den strukturellen Eigenschaften des Datensatzes zu passen. Sie reagiert robuster auf einzelne Ausreißer in den Merkmalen und scheint damit eine bessere Trennung zwischen den Klassen zu ermöglichen als die euklidische Distanz.

#### 3.3.4 k-Nearest-Neighbor mit Minkowski Metrik und p = 3

Die Minkowski Metrik kann über den Parameter p verschiedene Distanzen annehmen.
p = 3 deutet den Übergang zwischen euklidischen und Tschebyscheff-Abständen an (vgl. https://www.datacamp.com/de/tutorial/minkowski-distance).
Hierbei werden Abstände bei Größeren Unterschieden stärker betont.

In [ ]:
# Create k-NN model with k=10 and Minkowski distance metric (p=3)
knn_model = KNeighborsClassifier(n_neighbors=10, metric='minkowski', p=3)

# Train the model
knn_model.fit(X_train, y_train)

# Make predictions
y_pred_knn = knn_model.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
classification_rep_knn = classification_report(y_test, y_pred_knn)

# Print results
print(f"Model accuracy: {accuracy_knn:.2f}")
print(classification_rep_knn)


Das k-NN-Modell mit Minkowski-Distanz erreicht eine Genauigkeit von 77 % und liegt damit im oberen Leistungsbereich der bisher getesteten Verfahren.
Für Klasse 0 (keine KHK) ist der Recall mit 86 % besonders hoch, womit das Modell gesunde Fälle zuverlässig erkennt, obwohl die Präzision mit 67 % etwas niedriger ausfällt.
Klasse 1 (KHK) wird mit 87 % Präzision und 70 % Recall ebenfalls solide vorhergesagt.

Wie schon in der Beschreibung von [3.3.4](#334-k-nearest-neighbor-mit-minkowski-metrik-und-p-3) angesprochen, führt p = 3 zu einer Gewichtung, die größere Abstände stärker berücksichtigt als bei euklidischer oder Manhattan-Metrik. Dadurch kann das Modell empfindlicher auf auffällige Merkmalsunterschiede reagieren, welche jedoch auch nicht immer berechtigt sind, wenn man die Genauigkeit für p = 1 und p = 2 vergleicht.

Insgesamt bietet dieses Modell eine ausgewogene Performance, ähnlich wie die Varianten mit euklidischer oder Manhattan-Metrik.

### 3.4 Support Vector Machine

SVM ist ein supervised Klassifikationsverfahren, das eine Hyperebene mit dem größtmöglichen Abstand zwischen zwei Klassen findet.
Wichtig sind dabei die Support-Vektoren, die die Trennung bestimmen.
SVMs sind leistungsstark, aber oft schwer interpretierbar und rechenintensiv. Mit Soft-Margin können auch nicht perfekt trennbare Daten verarbeitet werden (vgl. Skript ML_07_Modelle-IV S.41-49).

In [ ]:
# Create SVM model with a linear kernel
svm_model = SVC(kernel='linear', random_state=42)

# Train the model
svm_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

# Print results
print(f"Model accuracy: {accuracy_svm:.2f}")
print(classification_rep_svm)


Das SVM-Modell erreicht eine Genauigkeit von 77 % und liefert damit ein gutes Ergebnis, vergleichbar mit AdaBoost und den besten k-NN-Varianten.
Die Klasse 0 (keine KHK) wird mit 79 % Recall gut erkannt, allerdings liegt die Präzision bei 69 %, was auf einige falsch positive Vorhersagen hinweist.
Klasse 1 (KHK) zeigt eine hohe Präzision von 83 % bei solidem Recall von 75 %. Das Modell identifiziert KHK-Fälle also sehr zuverlässig.

Da ein linearer Kernel verwendet wird, zeigt das gute Ergebnis, dass sich die Daten offenbar weitgehend linear trennen lassen, was auch durch die gute Balance von Präzision und Recall unterstrichen wird.
Gleichzeitig ist das Modell einfacher und weniger komplex zu trainieren als nichtlineare SVMs (vgl. https://www.ibm.com/think/topics/support-vector-machine).

### 3.5 Neuronales Netz

Ein neuronales Netz besteht aus mehreren Schichten künstlicher Neuronen, die über gewichtete Verbindungen Informationen weitergeben.
Ein Neuron wird aktiviert, wenn sein Output einen Schwellenwert überschreitet. So lernt das Netz, Muster in Daten zu erkennen (vgl. https://www.ibm.com/de-de/topics/neural-networks).

In [ ]:
def create_model(optimizer):
    # Define the model architecture
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # First hidden layer
        Dense(32, activation='relu'),  # Second hidden layer
        Dense(16, activation='relu'),  # Third hidden layer
        Dense(1, activation='sigmoid')  # Output layer (binary classification)
    ])

    # Compile the model
    model.compile(optimizer=optimizer,  # Set optimizer
                  loss='binary_crossentropy',  # Loss function for binary classification
                  metrics=['accuracy'])  # Metrics to track during training

    # Display model summary
    model.summary()
    return model


In [ ]:
# Create the model using the SGD optimizer
sgd_model = create_model(optimizer='sgd')

# Train the model
history_sgd = sgd_model.fit(X_train, y_train,
                            epochs=50,  # Number of epochs for training
                            batch_size=32,  # Batch size for training
                            validation_split=0.2,  # Split of training data for validation
                            verbose=1)  # Display progress during training

# Evaluate the model on the test set
test_loss_sgd, test_accuracy_sgd = sgd_model.evaluate(X_test, y_test)

# Visualize training history with Plotly

# Plot Accuracy
fig_accuracy = go.Figure()
fig_accuracy.add_trace(go.Scatter(x=list(range(1, 51)), y=history_sgd.history['accuracy'],
                                 mode='lines', name='Training Accuracy'))
fig_accuracy.add_trace(go.Scatter(x=list(range(1, 51)), y=history_sgd.history['val_accuracy'],
                                 mode='lines', name='Validation Accuracy'))

fig_accuracy.update_layout(
    title='Model Accuracy',
    xaxis_title='Epoch',
    yaxis_title='Accuracy'
)

# Plot Loss
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(x=list(range(1, 51)), y=history_sgd.history['loss'],
                             mode='lines', name='Training Loss'))
fig_loss.add_trace(go.Scatter(x=list(range(1, 51)), y=history_sgd.history['val_loss'],
                             mode='lines', name='Validation Loss'))

fig_loss.update_layout(
    title='Model Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)

# Show the figures
fig_accuracy.show()
fig_loss.show()

# Make predictions
y_pred_sgd = sgd_model.predict(X_test)
y_pred_classes_sgd = (y_pred_sgd > 0.5).astype(int)  # Convert probabilities to binary classes

# Classification Report
print(f"\nTest Accuracy: {test_accuracy_sgd:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_sgd))


**Grafische Analyse**

Die beiden dargestellten Graphen zeigen den Verlauf von Genauigkeit (Accuracy) und Verlust (Loss) über 50 Trainings-Epochen. In der oberen Grafik ist zu erkennen, dass sowohl die Trainingsgenauigkeit als auch die Validierungsgenauigkeit kontinuierlich ansteigen und sich gegen Ende des Trainings auf einem hohen Niveau bei ca. 84 % einpendeln. Dabei verlaufen beide Linien sehr nah beieinander, was ein Hinweis darauf ist, dass das Modell nicht overfitted und gut verallgemeinert.

In der unteren Grafik zum Loss zeigt sich ein analoges Bild: Sowohl der Trainings- als auch der Validierungsverlust sinken stetig, was auf eine stabile Optimierung des Modells hinweist. Auch hier verlaufen die Linien gleichmäßig und ohne plötzliche Ausschläge, was für ein ruhiges, zuverlässiges Lernverhalten spricht.

**Statistische Analyse**

Auf dem Testdatensatz erreicht das neuronale Netz eine Genauigkeit von 76,1 %, was mit den besten klassischen Verfahren wie SVM, AdaBoost oder k-NN vergleichbar ist.
Die Klasse 1 (KHK) wird besonders zuverlässig erkannt, mit einer Präzision von 81 % und einem Recall von 78 %. Das bedeutet, dass wenige gesunde Patienten fälschlich als krank eingestuft und auch die meisten tatsächlichen KHK-Fälle korrekt erkannt werden.

Die Klasse 0 (keine KHK) wird mit einer Präzision von 70 % und einem Recall von 74 % etwas schwächer, aber dennoch solide erfasst. Die F1-Scores von 72 % (Klasse 0) und 79 % (Klasse 1) deuten auf eine ausgewogene Gesamtleistung hin, ohne starke Verzerrung zugunsten einer Klasse.

In [ ]:
# Create the model using the Adam optimizer
adam_model = create_model(optimizer='adam')

# Train the model
history_adam = adam_model.fit(X_train, y_train,
                              epochs=50,  # Number of epochs for training
                              batch_size=32,  # Batch size for training
                              validation_split=0.2,  # Split of training data for validation
                              verbose=1)  # Display progress during training

# Evaluate the model on the test set
test_loss, test_accuracy_adam = adam_model.evaluate(X_test, y_test)

# Visualize training history with Plotly

# Plot Accuracy
fig_accuracy_adam = go.Figure()
fig_accuracy_adam.add_trace(go.Scatter(x=list(range(1, 51)), y=history_adam.history['accuracy'],
                                      mode='lines', name='Training Accuracy'))
fig_accuracy_adam.add_trace(go.Scatter(x=list(range(1, 51)), y=history_adam.history['val_accuracy'],
                                      mode='lines', name='Validation Accuracy'))

fig_accuracy_adam.update_layout(
    title='Model Accuracy',
    xaxis_title='Epoch',
    yaxis_title='Accuracy'
)

# Plot Loss
fig_loss_adam = go.Figure()
fig_loss_adam.add_trace(go.Scatter(x=list(range(1, 51)), y=history_adam.history['loss'],
                                  mode='lines', name='Training Loss'))
fig_loss_adam.add_trace(go.Scatter(x=list(range(1, 51)), y=history_adam.history['val_loss'],
                                  mode='lines', name='Validation Loss'))

fig_loss_adam.update_layout(
    title='Model Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)

# Show the figures
fig_accuracy_adam.show()
fig_loss_adam.show()

# Make predictions
y_pred_adam = adam_model.predict(X_test)
y_pred_classes_adam = (y_pred_adam > 0.5).astype(int)  # Convert probabilities to binary classes

# Classification Report
print(f"\nTest Accuracy: {test_accuracy_adam:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_adam))


**Grafische Analyse**

In der oberen Grafik ist der Verlauf der Trainings- und Validierungsgenauigkeit zu sehen. Das Modell erreicht bereits nach wenigen Epochen sehr gute Werte, wobei die Trainingsgenauigkeit weiter kontinuierlich ansteigt, während sich die Validierungsgenauigkeit ab etwa Epoche 10 auf einem Plateau bei ca. 84 % einpendelt. Die Kurven beginnen ab dann deutlich auseinanderzulaufen, was ein Hinweis auf Overfitting ist.

In der unteren Grafik erkennt man, dass der Trainings-Loss stetig sinkt, der Validierungs-Loss jedoch ab etwa Epoche 10 nicht weiter fällt, sondern später sogar leicht ansteigt. Auch das deutet darauf hin, dass das Modell beginnt, sich zu stark auf die Trainingsdaten zu spezialisieren und die Generalisierung auf neue Daten nicht mehr im Fokus steht.

Ein früheres Stoppen (Early Stopping )wäre sinnvoll, um das beginnende Overfitting zu verhindern.

**Statistische Analyse**

Das Modell erzielt genau wie das vorherige Modell auf den Testdaten eine Genauigkeit von 76,1 %, was ein solides Ergebnis darstellt. Die Leistung ist damit ebenfalls vergleichbar mit anderen starken Verfahren wie SVM oder AdaBoost.
Klasse 0 (keine KHK) wird mit 71 % Präzision und Recall erkannt, Klasse 1 (KHK) mit jeweils 79 %. Das Modell erfasst KHK-Fälle somit zuverlässiger als gesunde.

## 4. Bedeutung der einzelnen Features

### 4.1 Feature-Bedeutung von PCA

In [ ]:
# Get the feature names (excluding the target variable "KHK")
feature_names = data.columns.tolist()
feature_names.remove("KHK")

# Compute the importance of features from PCA components
feature_importance = np.abs(pca.components_).sum(axis=0)

# Create DataFrame for Plotly visualization
df_plot = pd.DataFrame({"Feature": feature_names, "Wichtigkeit": feature_importance})

# Create an interactive bar plot with Plotly
fig = px.bar(df_plot, x="Feature", y="Wichtigkeit", title="Feature Importance from PCA", labels={"Feature": "Feature", "Wichtigkeit": "Feature Importance"})
fig.update_xaxes()  # Update x-axis for better readability
fig.show()


### 4.2 Feature-Bedeutung für Random Forest

In [ ]:
# Get the feature importance from the Random Forest model
feature_importance = clf.feature_importances_

# Create DataFrame for Plotly visualization
df_plot = pd.DataFrame({"Feature": X.columns.tolist(), "Wichtigkeit": feature_importance})

# Create an interactive bar plot with Plotly
fig = px.bar(df_plot, x="Feature", y="Wichtigkeit", title="Feature Importance from Random Forest", labels={"Feature": "Feature", "Wichtigkeit": "Feature Importance"})
fig.update_xaxes()  # Update x-axis for better readability
fig.show()


### 4.3 Feature Bedeutung SVM

In [ ]:
# Get the absolute values of the coefficients as feature importance
feature_importance = abs(svm_model.coef_).flatten()

# Create DataFrame for Plotly visualization
df_plot = pd.DataFrame({"Feature": X.columns.tolist(), "Wichtigkeit": feature_importance})

# Create an interactive bar plot with Plotly
fig = px.bar(df_plot, x="Feature", y="Wichtigkeit", title="Feature Importance from SVM", labels={"Feature": "Feature", "Wichtigkeit": "Feature Importance"})
fig.show()


## 5. Feature-Engineering

Für das Feature Engineering wurden zwei Klassifikationsverfahren ausgewählt. Einmal wurde k-Nearest-Neighbor mit Manhattan Metrik genutzt und zusätzlich klassische Entscheidungsbäume. Diese beiden Klassifikationsverfahren wurden ausgewählt, dass k-Nearest-Neighbor mit Manhattan Metrik beim testen die höchste und klassische Entscheidungsbäume die schlechteste Genauigkeit hatten. 

### 5.1 Generieren der PCA-Hauptkomponenten Daten

In [ ]:
# Define the number of principal components to keep (can be adjusted)
pca_components = 2

# Perform PCA transformation with the specified number of components
pca = PCA(n_components=pca_components)
X_pca = pca.fit_transform(data_scaled)

# Convert the PCA results into a DataFrame
df_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(pca_components)])

# Add the target variable "KHK" to the PCA DataFrame
df_pca['KHK'] = data['KHK'].values

# Split the data into training and test sets
X_train_pca, X_test_pca, y_train, y_test = train_test_split(df_pca.drop(columns=["KHK"]), df_pca["KHK"], test_size=0.2, random_state=42)


### 5.2 Testen des Feature-Engineering auf k-Nearest-Neighbor mit Manhattan Metrik

In [ ]:
# Create k-NN model with k=10 for PCA features using Manhattan distance
knn_model_pca = KNeighborsClassifier(n_neighbors=10, metric='manhattan')

# Train the model on PCA-transformed features
knn_model_pca.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred_knn_pca = knn_model_pca.predict(X_test_pca)

# Calculate accuracy
accuracy_knn_pca = accuracy_score(y_test, y_pred_knn_pca)
classification_rep_knn_pca = classification_report(y_test, y_pred_knn_pca)

# Print accuracy and classification report
print(f"Model accuracy: {accuracy_knn_pca:.2f}")
print(classification_rep_knn_pca)


Das k-NN-Modell mit Manhattan-Distanz und PCA-Reduktion auf zwei Komponenten erzielt eine Genauigkeit von 79 %, womit es mit den besten bisher getesteten Modellen mithalten kann.
Besonders auffällig ist die Leistung bei Klasse 0 (keine KHK) mit einem Recall von 92 %, was bedeutet, dass fast alle gesunden Fälle korrekt erkannt werden. Die Präzision liegt bei 69 %, was auf einige falsch-positive Vorhersagen hinweist.
Klasse 1 (KHK) wird mit einer sehr hohen Präzision von 93 % und einem Recall von 70 % vorhergesagt. Das Modell identifiziert also KHK-Fälle sehr zuverlässig, übersieht jedoch auch einen Teil davon.

Durch die PCA wurde die Dimensionalität stark reduziert. Trotz dieser Vereinfachung bleibt die Leistung auf hohem Niveau, was zeigt, dass die wichtigsten Informationen im Datensatz gut durch die Hauptkomponenten abgebildet werden.
Somit ist die anfängliche Aussage, PCA wäre hinsichtlich der visuellen Analyse wenig aussagekräftig, hinfällig.

### 5.3 Testen des Feature-Engineering auf einem klassischen Entscheidungsbaum 

In [ ]:
# Create Decision Tree model for PCA features
clf_tree_pca = DecisionTreeClassifier(random_state=42)

# Train the model on PCA-transformed features
clf_tree_pca.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred_tree_pca = clf_tree_pca.predict(X_test_pca)

# Calculate accuracy
accuracy_tree_pca = accuracy_score(y_test, y_pred_tree_pca)
classification_rep_tree_pca = classification_report(y_test, y_pred_tree_pca)

# Print accuracy and classification report
print(f"Model accuracy: {accuracy_tree_pca:.2f}")
print(classification_rep_tree_pca)


Das Eklassische ntscheidungsbaum-Modell auf den PCA-transformierten Daten erreicht eine Genauigkeit von 70 % und liegt damit deutlich unterhalb des k-NN-Modells mit gleicher Datenbasis.

Klasse 0 (keine KHK) wird mit 62 % Präzision und 71 % Recall erkannt – gesunde Fälle werden also mehrheitlich gefunden, aber mit vergleichsweise vielen Fehlalarmen.
Klasse 1 (KHK) zeigt mit 77 % Präzision und 68 % Recall etwas bessere Balance, allerdings auf Kosten einer geringeren Sensitivität.

Insgesamt wirkt das Modell instabiler, was bei Entscheidungsbäumen typisch ist, besonders bei reduzierter Datenkomplexität wie nach PCA:

"Unstable: Decision trees can be unstable, as small changes in the data can result in significant changes to the tree structure." (vgl. https://lazyprogrammer.me/mlcompendium/supervised/decision_trees.html)